In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format, lit, concat
import re # Adicionado a importação de re

# --- INÍCIO: FUNÇÃO AUXILIAR SNAKE_CASE ---
def to_snake_case(name):
    """
    Converte uma string (CamelCase, UPPER_CASE, etc.) para snake_case.
    Ex: "DataRegistro" -> "data_registro"
    Ex: "ATENDIMENTOS_SEXO" -> "atendimentos_sexo"
    Ex: "CLIMA_TEMPO_PM2_5" -> "clima_tempo_pm2_5"
    """
    # Insere _ antes de letras maiúsculas (ex: DataRegistro -> Data_Registro)
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    # Insere _ antes de grupos de letras maiúsculas/números (ex: CBO, PM2_5)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    # Converte tudo para minúsculo
    return s2.lower()
# --- FIM: FUNÇÃO AUXILIAR SNAKE_CASE ---

# =======================================================================
# 1. CONFIGURAÇÃO E INICIALIZAÇÃO DO SPARK
# =======================================================================
conf = SparkConf()
conf.set(
    "spark.jars.packages",
    "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901"
)
conf.set(
    "spark.hadoop.fs.s3a.aws.credentials.provider",
    "com.amazonaws.auth.InstanceProfileCredentialsProvider"
)

spark = SparkSession.builder.appName("ExternalDataIntegration").config(conf=conf).getOrCreate()

# =======================================================================
# 2. DEFINIÇÃO DOS CAMINHOS DOS ARQUIVOS DE ENTRADA E SAÍDA
# =======================================================================

# Arquivos de Entrada no bucket trusted
# (Mantidos os nomes dos arquivos conforme o script original)
INPUT_PATHS = {
    "atendimentos": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv",
    "clima_tempo": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv",
    "febre_amarela": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv",
    "sindrome_gripal": "s3a://bucket-trusted-upa-connect-sofh/BasesExternas/SindromeGripal/sindromeGripalTratada.csv",
}

# Caminho de Saída no bucket client
FINAL_OUTPUT_DIR = "s3a://bucket-client-upa-connect-sofh/basesExternas"
FINAL_FILENAME = "BasesExternasIntegradas.csv"
TEMP_STAGING_DIR = f"{FINAL_OUTPUT_DIR}/_temp_staging_integrated"
FINAL_OUTPUT_PATH = f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}" # Adicionado: Caminho completo do arquivo final

# Coluna de Junção Comum (AGORA COM HORA INCLUÍDA)
JOIN_COLUMN = "DATA_HORA"

# =======================================================================
# 2.1. EXCLUSÃO DO ARQUIVO FINAL ANTERIOR
# =======================================================================

print(f"Verificando e excluindo o arquivo final anterior em: {FINAL_OUTPUT_PATH}")
try:
    # Acessa as classes Java do Hadoop para manipulação de arquivos no S3
    Path = spark._jvm.org.apache.hadoop.fs.Path
    hadoop_conf = spark._jsc.hadoopConfiguration()
    fs = Path(FINAL_OUTPUT_PATH).getFileSystem(hadoop_conf)

    if fs.exists(Path(FINAL_OUTPUT_PATH)):
        # Tenta excluir o arquivo (não recursivo, pois é esperado que seja um arquivo)
        fs.delete(Path(FINAL_OUTPUT_PATH), False)
        print(f"Arquivo anterior {FINAL_FILENAME} excluído com sucesso.")
    else:
        print(f"Arquivo anterior {FINAL_FILENAME} não encontrado. Prosseguindo.")

except Exception as e:
    print(f"\nATENÇÃO: Não foi possível verificar/excluir o arquivo final no S3. O script pode falhar na etapa de salvar/renomear: {e}")

# =======================================================================
# 3. LEITURA DOS DADOS E PREPARAÇÃO PARA JOIN (COM DATA E HORA)
# =======================================================================

dataframes = {}

print("Iniciando a leitura e padronização dos DataFrames do bucket trusted...")
for name, path in INPUT_PATHS.items():
    print(f"Lendo {name} de: {path}")
    
    # Define as opções de leitura padrão
    read_options = {
        "header": "true",
        "encoding": "UTF-8"
    }

    # Define delimitador específico.
    if name in ["atendimentos", "clima_tempo", "febre_amarela", "sindrome_gripal"]:
        read_options["delimiter"] = ";"
        
    # Leitura de CSV
    df = spark.read.options(**read_options).csv(path)
    
    # --- NOVO BLOCO: Padronização da Coluna de Data/Hora (JOIN_COLUMN) ---
    
    # As bases "atendimentos" e "clima_tempo" possuem a coluna DATA_ISO (Data e Hora)
    if name in ["atendimentos", "clima_tempo"]:
        # Usa DATA_ISO (ISO 8601) diretamente no formato YYYY-MM-DD HH:MM:SS
        df = df.withColumn(
            JOIN_COLUMN,
            date_format(col("DATA_ISO"), "yyyy-MM-dd HH:mm:ss") # Mantém a hora
        ).drop("DATA_ISO")
        
    # As bases "febre_amarela" e "sindrome_gripal" são diárias (SÓ DATA)
    elif name == "febre_amarela":
        # 1. Converte a coluna DT_IS (DD/MM/YYYY) para a data padronizada (YYYY-MM-DD)
        df = df.withColumn(
            "DATA_PADRAO",
            date_format(to_date(col("DT_IS"), "dd/MM/yyyy"), "yyyy-MM-dd")
        ).drop("DT_IS")
        
        # 2. Concatena com a hora '00:00:00'
        df = df.withColumn(
            JOIN_COLUMN,
            concat(col("DATA_PADRAO"), lit(" 00:00:00")) # Adiciona a hora 00:00:00
        ).drop("DATA_PADRAO")
        
    elif name == "sindrome_gripal":
        # DATA_NOTIFICACAO_TRATADA já está em YYYY-MM-DD (tratada anteriormente)
        # 1. Renomeia temporariamente para DATA_PADRAO
        df = df.withColumnRenamed("DATA_NOTIFICACAO_TRATADA", "DATA_PADRAO")
        
        # 2. Concatena com a hora '00:00:00'
        df = df.withColumn(
            JOIN_COLUMN,
            concat(col("DATA_PADRAO"), lit(" 00:00:00")) # Adiciona a hora 00:00:00
        ).drop("DATA_PADRAO")
        
        
    # --- FIM NOVO BLOCO ---
        
    # Renomeia colunas para evitar conflitos (exceto a coluna de junção)
    for c in df.columns:
        if c.upper() != JOIN_COLUMN.upper():
            # Aplica o prefixo em caixa alta (ex: ATENDIMENTOS_ID_PACIENTE)
            # Nota: O nome 'id' da sindrome gripal vira 'SINDROME_GRIPAL_id'
            # Isso será corrigido pela função snake_case no final
            df = df.withColumnRenamed(c, f"{name.upper()}_{c}")
            
    # Seleciona o DataFrame com a nova coluna de junção
    dataframes[name] = df

print("Leitura e preparação concluídas.")
print(f"Chave de junção padronizada (data e hora): {JOIN_COLUMN}")

# =======================================================================
# 4. REALIZAÇÃO DA JUNÇÃO (FULL OUTER JOIN)
# =======================================================================
print("\nIniciando a junção dos DataFrames...")

# Começa com o DataFrame de Atendimentos
df_base = dataframes["atendimentos"]

# Realiza a junção com os demais DataFrames
for name, df_join in dataframes.items():
    if name != "atendimentos":
        # Full outer join para manter a maior granularidade possível (a data e hora)
        df_base = df_base.join(df_join, on=JOIN_COLUMN, how="full_outer")
        print(f"Junção com {name} concluída.")

print("Junção de todos os DataFrames concluída.")

# =======================================================================
# 5. REMOÇÃO DE COLUNAS DE ID E PADRONIZAÇÃO FINAL PARA SNAKE_CASE
# =======================================================================

# 1. REMOÇÃO DE COLUNAS DE ID
# Colunas a serem removidas do resultado final
# **** CORREÇÃO APLICADA AQUI ****
# Mantido SINDROME_GRIPAL_id (que virou SINDROME_GRIPAL_ID ou SINDROME_GRIPAL_id)
# Removido apenas FEBRE_AMARELA_ID
COLUNAS_PARA_REMOVER = [
    "FEBRE_AMARELA_ID"
]

# Cria uma lista de colunas para manter, excluindo as colunas de ID
colunas_para_renomear = [c for c in df_base.columns if c.upper() not in COLUNAS_PARA_REMOVER]

# Aplica a seleção
df_base = df_base.select(colunas_para_renomear)

# 2. PADRONIZAÇÃO PARA SNAKE_CASE
print("\nPadronizando nomes das colunas para snake_case...")

# Cria um mapa de colunas antigas para novas (snake_case)
col_rename_map = {c: to_snake_case(c) for c in df_base.columns}

# Aplica a renomeação a todas as colunas
for old_name, new_name in col_rename_map.items():
    df_base = df_base.withColumnRenamed(old_name, new_name)

# Renomeia a coluna de junção para 'data_hora' (que será 'data_hora' em snake_case)
# Se o JOIN_COLUMN já for "DATA_HORA", ele vira "data_hora" aqui.
df_base.printSchema()
print(f"Total de linhas na base integrada: {df_base.count()}")

# =======================================================================
# 6. SALVANDO E RENOMEANDO O RESULTADO NO S3 (BUCKET CLIENT)
# =======================================================================

# 1. Escreve o resultado no caminho temporário
print(f"\nEscrevendo dados temporariamente em: {TEMP_STAGING_DIR}")

# NOTA: Coalesce(1) para garantir a geração de um único arquivo CSV.
df_base.coalesce(1).write \
    .option('delimiter', ';') \
    .option('header', 'true') \
    .option('encoding', 'UTF-8') \
    .mode('overwrite') \
    .csv(TEMP_STAGING_DIR)

# 2. Renomeia o arquivo gerado
try:
    # Acessa a classe 'Path' da JVM através do gateway do Spark
    Path = spark._jvm.org.apache.hadoop.fs.Path
    
    # Acessa a configuração do Hadoop
    hadoop_conf = spark._jsc.hadoopConfiguration()
    
    # Obtém o objeto FileSystem para o caminho temporário
    fs = Path(TEMP_STAGING_DIR).getFileSystem(hadoop_conf)

    # Encontra o arquivo gerado (part-00000-*.csv) dentro do diretório temporário
    list_status = fs.globStatus(Path(TEMP_STAGING_DIR + "/part-00000-*.csv"))

    if list_status:
        # Pega o caminho completo do arquivo gerado
        generated_file_path = list_status[0].getPath()

        # Define o caminho final e o nome específico para o arquivo
        final_output_path = Path(f"{FINAL_OUTPUT_DIR}/{FINAL_FILENAME}")

        # Renomeia (move) o arquivo para o caminho e nome definitivos
        fs.rename(generated_file_path, final_output_path)
        
        # 3. Deleta o diretório temporário (que ficou vazio) e outros arquivos de metadados
        fs.delete(Path(TEMP_STAGING_DIR), True)
        
        print(f"\n✅ Base integrada salva e renomeada com sucesso para: {final_output_path}")

    else:
        print("\nErro: Não foi possível encontrar o arquivo CSV gerado (part-00000-*.csv) no caminho temporário.")

except Exception as e:
    print(f"\nOcorreu um erro durante a renomeação do arquivo no S3: {e}")

# Encerra a sessão Spark
spark.stop()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b17a9880-e1a2-421c-98df-ac59f15da9ed;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 512ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

Verificando e excluindo o arquivo final anterior em: s3a://bucket-client-upa-connect-sofh/basesExternas/BasesExternasIntegradas.csv


25/11/01 23:49:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Arquivo anterior BasesExternasIntegradas.csv excluído com sucesso.
Iniciando a leitura e padronização dos DataFrames do bucket trusted...
Lendo atendimentos de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/Atendimentos/AtendimentosTratada.csv


Lendo clima_tempo de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/ClimaTempo/ClimaTempoTratada.csv
Lendo febre_amarela de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/FebreAmarela/FebreAmarelaTratada.csv
Lendo sindrome_gripal de: s3a://bucket-trusted-upa-connect-sofh/BasesExternas/SindromeGripal/sindromeGripalTratada.csv
Leitura e preparação concluídas.
Chave de junção padronizada (data e hora): DATA_HORA

Iniciando a junção dos DataFrames...
Junção com clima_tempo concluída.
Junção com febre_amarela concluída.
Junção com sindrome_gripal concluída.
Junção de todos os DataFrames concluída.

Padronizando nomes das colunas para snake_case...
root
 |-- data_hora: string (nullable = true)
 |-- atendimentos_sexo: string (nullable = true)
 |-- atendimentos_tipo_de_unidade: string (nullable = true)
 |-- atendimentos_descricao_do_procedimento: string (nullable = true)
 |-- atendimentos_descricao_do_cbo: string (nullable = true)
 |-- atendimentos_nacionalidade: string (nullable =

Total de linhas na base integrada: 1307095

Escrevendo dados temporariamente em: s3a://bucket-client-upa-connect-sofh/basesExternas/_temp_staging_integrated


25/11/01 23:50:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/11/01 23:50:40 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/11/01 23:50:41 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.



✅ Base integrada salva e renomeada com sucesso para: s3a://bucket-client-upa-connect-sofh/basesExternas/BasesExternasIntegradas.csv
